# A/B Testing Course

## Lesson 1. Basic Statistics

### Homework

#### Import Libraries

In [84]:
import os
import pandas as pd
import numpy as np

#### Import Data

In [2]:
URL_BASE = ''

def read_database(file_name):
    return pd.read_csv(os.path.join(URL_BASE, file_name))

In [3]:
df_sales = read_database('2022-04-01T12_df_sales.csv')
df_web_logs = read_database('2022-04-01T12_df_web_logs.csv')

df_sales['date'] = pd.to_datetime(df_sales['date'])
df_web_logs['date'] = pd.to_datetime(df_web_logs['date'])

In [13]:
df_sales_details = pd.read_csv('df_sales_detail.csv')

#### Data Description

df_sales - information about purchases, one row represents one order:
- sale_id - purchase identifier;
- date - date of purchase;
- count_pizza - number of pizzas in the order;
- count_drink - number of drinks in the order;
- price - order price;
- user_id - user identifier.   

df_sales_details - information about menu:
- sale_id - purchase identifier;
- good - position on menu;
- price - position price;
- date - purchase date;
- user_id - user identifier.  

df_web_logs - a table with logs of website visits:
- user_id - user identifier;
- page - which page of the website was opened. Page types:
    - m - menu page;
    - b - cart and order confirmation page;
    - p - payment page;
- date - date of the page opening;
- load_time - page load time in milliseconds.

#### Checking Data

In [14]:
df_sales.head()

,sale_id,date,count_pizza,count_drink,price,user_id
0,1000001,2022-02-04 10:00:24,1,0,720,1c1543
1,1000002,2022-02-04 10:02:28,1,1,930,a9a6e8
2,1000003,2022-02-04 10:02:35,3,1,1980,23420a
3,1000004,2022-02-04 10:03:06,1,1,750,3e8ed5
4,1000005,2022-02-04 10:03:23,1,1,870,cbc468


In [8]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203847 entries, 0 to 203846
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   sale_id      203847 non-null  int64         
 1   date         203847 non-null  datetime64[ns]
 2   count_pizza  203847 non-null  int64         
 3   count_drink  203847 non-null  int64         
 4   price        203847 non-null  int64         
 5   user_id      203847 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 9.3+ MB


In [7]:
df_web_logs.head()

,user_id,page,date,load_time
0,f25239,m,2022-02-03 23:45:37,80.8
1,06d6df,m,2022-02-03 23:49:56,70.5
2,06d6df,m,2022-02-03 23:51:16,89.7
3,f25239,m,2022-02-03 23:51:43,74.4
4,697870,m,2022-02-03 23:53:12,66.8


In [10]:
df_web_logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1964507 entries, 0 to 1964506
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   user_id    object        
 1   page       object        
 2   date       datetime64[ns]
 3   load_time  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 60.0+ MB


In [15]:
df_sales_details.head()

,sale_id,good,price,date,user_id
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8
3,1000003,cheese pizza,600,2022-02-04 10:02:35,23420a
4,1000003,italian pizza,720,2022-02-04 10:02:35,23420a


In [16]:
df_sales_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416481 entries, 0 to 416480
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   sale_id  416481 non-null  int64 
 1   good     416481 non-null  object
 2   price    416481 non-null  int64 
 3   date     416481 non-null  object
 4   user_id  416481 non-null  object
dtypes: int64(2), object(3)
memory usage: 15.9+ MB


#### Task 1. 

Determine the top 3 products by total revenue.

In [23]:
df_sales_details.groupby('good', as_index=False) \
    .agg({'price': 'sum'}) \
    .sort_values('price', ascending=False) \
    .reset_index(drop=True) \
    .head(3)

,good,price
0,chefs pizza,24558240
1,double pepperoni pizza,22558380
2,chicken bbq pizza,17622540


#### Task 2. 

Estimate the average time it takes from a user's site visit to making a purchase. We will consider a site visit to be a purchase if it occurred no earlier than two hours before the purchase was made. In other words, for each purchase, we need to calculate the time elapsed from the purchase to the first site visit by the same user within two hours before the purchase. Then, calculate the average of these values.  

Convert the result to minutes and round to the nearest integer value.

In [115]:
purch_delta_s = []
users_lst = list(set(df_sales.user_id.to_list()))

cnt = 0
for user in users_lst[0:]:
    dates_lst = df_sales[df_sales['user_id'] == user].date.to_list()
    df_user = df_web_logs[df_web_logs['user_id'] == user]
    for d in dates_lst:
        finish = d
        start = d - pd.Timedelta(hours=2)
        df_temp = df_user[(df_user['date'] <= finish) & (df_user['date'] >= start)]
        df_temp = df_temp.sort_values('date')
        ans = pd.to_datetime(finish) - pd.to_datetime(df_temp.head(1).date.values)
        purch_delta_s.append(ans.seconds[0])
    cnt += 1
    
    if cnt % 10000 == 0:
        print(f'{cnt} users have been processed.')
    
    
print(f'A number of purchases processed: {len(purch_delta_s):,.0f}')
print(f'An average time to make one purchase is: {round(np.mean(purch_delta_s)/60, 0):.0f} minutes')

10000 users have been processed.
20000 users have been processed.
30000 users have been processed.
40000 users have been processed.
50000 users have been processed.
60000 users have been processed.
70000 users have been processed.
80000 users have been processed.
90000 users have been processed.
A number of purchases processed: 203,847
An average time to make one purchase is: 17 minutes


#### Task 3. 

In [ ]:
tbc..